In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline

/home/utk/.virtualenvs/jupjup/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/utk/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
# готовим ТРЕНИРОВОЧНЫЕ данные и метки для них
data_train = pd.read_csv('products_sentiment_train.tsv', header=None, sep='\t')
# data_train.head()
texts_train = data_train[0].values
labels = data_train[1].values

In [6]:
stemmer = SnowballStemmer('english') # подключаем английский словарь для Стемминга
wnl = WordNetLemmatizer() # лемматизация

texts_train2 = []

# лемматизация 
for review in texts_train:
    tokens = nltk.word_tokenize(review)
    texts_train2.append([wnl.lemmatize(token) for token in tokens]) # лемматизация
    
# сшиваем строки в список
texts_train2 = list(map(lambda x: ' '.join(x), texts_train2))

In [8]:
import pickle

# Векторайзер и модель
tf_vectorizer = TfidfVectorizer(ngram_range=(3, 16), analyzer=u'char')
mx = tf_vectorizer.fit_transform(texts_train2)
model = LogisticRegression(class_weight='balanced')
model.fit(mx, labels)

# сериализация векторайзера и модели в бинарные файлы
with open('./DefaultLogisticBigramUnprocessedTextSentiment.pkl', 'wb') as handle:
    pickle.dump(model, handle)
with open('./BigramUnprocessedVectorizer.pkl', 'wb') as handle:
    pickle.dump(tf_vectorizer, handle)

In [17]:
from my_sentiment_classifier import MySentimentClassifier
from codecs import open
import time
from flask import Flask, render_template, request

app = Flask(__name__)

print ("Preparing classifier")
start_time = time.time()
classifier = MySentimentClassifier()
print ("Classifier is ready")
print (time.time() - start_time, "seconds")

@app.route("/", methods=["POST", "GET"])
def index_page(text="", prediction_message=""):
    if request.method == "POST":
        text = request.form["text"]
        print (text)
        prediction_message = classifier.get_prediction_message(text)
        print (prediction_message)
    return render_template('hello.html', text=text, prediction_message=prediction_message)

if __name__ == "__main__":
    app.run()

ImportError: cannot import name 'MySentimentClassifier'